In [2]:
import tensorflow as tf
from tensorflow import keras
import glob

import numpy as np
import matplotlib.pyplot as plt
import os

from PIL import Image

In [214]:
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
# 	print(len(images), " images to be resized.")

    # 파일마다 모두 56X56 사이즈로 바꾸어 저장합니다.
	target_size=(56,56)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")
	
# 트래인 파일 변경
# 가위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME") + "/aiffel/EX/EX1/train_data/scissor"
resize_images(image_dir_path)

# 바위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
img_dir_rock = os.getenv("HOME")+"/aiffel/EX/EX1/train_data/rock"
resize_images(img_dir_rock)

# 보 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
img_dir_paper = os.getenv("HOME")+"/aiffel/EX/EX1/train_data/paper"
resize_images(img_dir_paper)

# 테스트 사이즈 파일 변경
img_dir_paper = os.getenv("HOME")+"/aiffel/EX/EX1/test_data/scissor"
resize_images(img_dir_paper)
img_dir_paper = os.getenv("HOME")+"/aiffel/EX/EX1/test_data/rock"
resize_images(img_dir_paper)
img_dir_paper = os.getenv("HOME")+"/aiffel/EX/EX1/test_data/paper"
resize_images(img_dir_paper)

1000  images resized.
1001  images resized.
1000  images resized.
200  images resized.
200  images resized.
200  images resized.


In [215]:
def load_data(img_path, number_of_data=3001):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=56
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/aiffel/EX/EX1/train_data/"
(x_train, y_train)=load_data(image_dir_path)
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 3001 입니다.
x_train shape: (3001, 56, 56, 3)
y_train shape: (3001,)


In [216]:
# x_test, y_test를 만드는 방법은 x_train, y_train을 만드는 방법과 아주 유사합니다.
def load_test_data(img_path, number_of_data=600):
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=56
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels


test_path = os.getenv("HOME")+"/aiffel/EX/EX1/test_data/"
(x_test, y_test)=load_test_data(test_path)
x_test_norm = x_test/255.0

print("x_test shape: {}".format(x_test.shape))
print("y_test shape: {}".format(y_test.shape))

학습데이터(x_train)의 이미지 개수는 600 입니다.
x_test shape: (600, 56, 56, 3)
y_test shape: (600,)


In [217]:
# model을 직접 만들어 보세요.
# Hint! model의 입력/출력부에 특히 유의해 주세요. 가위바위보 데이터셋은 MNIST 데이터셋과 어떤 점이 달라졌나요?
n_channel_1=128
n_channel_2=128
n_dense=64

model=keras.models.Sequential()
model.add(keras.layers.AveragePooling2D(6, (3,3), input_shape=(56,56,3)))
model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(56,56,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(n_dense, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))


model.summary()

Model: "sequential_53"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
average_pooling2d_35 (Averag (None, 17, 17, 3)         0         
_________________________________________________________________
conv2d_114 (Conv2D)          (None, 15, 15, 128)       3584      
_________________________________________________________________
max_pooling2d_106 (MaxPoolin (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_115 (Conv2D)          (None, 5, 5, 128)         147584    
_________________________________________________________________
max_pooling2d_107 (MaxPoolin (None, 2, 2, 128)         0         
_________________________________________________________________
flatten_49 (Flatten)         (None, 512)               0         
_________________________________________________________________
dense_103 (Dense)            (None, 64)              

In [218]:
# model을 학습시키는 코드를 직접 작성해 보세요.
# Hint! model.compile()과 model.fit()을 사용해 봅시다.

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train_norm, y_train, epochs=20) #, batch_size=10

Epoch 1/20
94/94 [==============================] - 1s 3ms/step - loss: 1.0945 - accuracy: 0.3729
Epoch 2/20
94/94 [==============================] - 0s 3ms/step - loss: 1.0363 - accuracy: 0.4482
Epoch 3/20
94/94 [==============================] - 0s 3ms/step - loss: 0.9156 - accuracy: 0.5428
Epoch 4/20
94/94 [==============================] - 0s 3ms/step - loss: 0.8097 - accuracy: 0.6275
Epoch 5/20
94/94 [==============================] - 0s 3ms/step - loss: 0.7077 - accuracy: 0.7051
Epoch 6/20
94/94 [==============================] - 0s 3ms/step - loss: 0.6191 - accuracy: 0.7468
Epoch 7/20
94/94 [==============================] - 0s 3ms/step - loss: 0.5322 - accuracy: 0.7884
Epoch 8/20
94/94 [==============================] - 0s 3ms/step - loss: 0.4811 - accuracy: 0.8174
Epoch 9/20
94/94 [==============================] - 0s 3ms/step - loss: 0.4216 - accuracy: 0.8354
Epoch 10/20
94/94 [==============================] - 0s 3ms/step - loss: 0.3807 - accuracy: 0.8567
Epoch 11/20
94/94 [

In [219]:
# model을 학습시키는 코드를 직접 작성해 보세요.
# Hint! model.evaluate()을 사용해 봅시다.

test_loss, test_accuracy = model.evaluate(x_test_norm,y_test,verbose=2)

print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

19/19 - 0s - loss: 5.8070 - accuracy: 0.4717
test_loss: 5.807046413421631 
test_accuracy: 0.4716666638851166


In [124]:
# 지금은 보고 아 그렇구나 하는 상태인것 같습니다. 
# 용어나 accuracy 조절이 잘 안되는 부분, 문서를 봐도 영어에 내용도 많다보니 어려웠습니다. 
# loss가 높고(x00.00) accuracy가 0.0X대가 나와서 중간에 퍼실님께 놓친 부분 피드백 받아서 막힌 부분을 알 수 있어서 다행이었습니다. 
# 자신의 코드만 보고있다보면 보일 부분도 안 보일 때가 있는데, 이럴땐 다른 분이 봐주는 것도 좋은것 같습니다.
# 데이터 추가하고, 수치들 변경, 모델 변경을 했지만 잘 안됐습니다.
# 데이터를 더 명확한 것으로 사용을 해야할지 생각되고, 모델에 대한 이해가 부족합니다.
# 앞으로도 계속 이런 과정이 있을텐데 겁먹지 말고 집중해서 해야겠습니다.
